In [2]:
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster()
import numpy as np
import pandas as pd
import cudf
import os
#from cuml import LinearRegression as cuLinearRegression
from sklearn.linear_model import LinearRegression as skLinearRegression
from sklearn.datasets import make_regression
import dask_cuml
import dask_cudf

/conda/envs/rapids/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/conda/envs/rapids/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
/conda/envs/rapids/lib/python3.7/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


# Helper Functions

In [3]:
from timeit import default_timer

class Timer(object):
    def __init__(self):
        self._timer = default_timer
    
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        """Start the timer."""
        self.start = self._timer()

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        self.interval = self.end - self.start
        

In [4]:
import gzip
def load_data(nrows, ncols, cached = 'data/mortgage.npy.gz'):
    #split the dataset in a 80:20 split
    train_rows = int(nrows*0.8)
    if os.path.exists(cached):
        print('use mortgage data')

        with gzip.open(cached) as f:
            X = np.load(f)
        # the 4th column is 'adj_remaining_months_to_maturity'
        # used as the label
        X = X[:,[i for i in range(X.shape[1]) if i!=4]]
        y = X[:,4:5]
        rindices = np.random.randint(0,X.shape[0]-1,nrows)
        X = X[rindices,:ncols]
        y = y[rindices]
        df_y_train = pd.DataFrame({'fea%d'%i:y[0:train_rows,i] for i in range(y.shape[1])})
        df_y_test = pd.DataFrame({'fea%d'%i:y[train_rows:,i] for i in range(y.shape[1])})
    else:
        print('use random data')
        X,y = make_regression(n_samples=nrows,n_features=ncols,n_informative=ncols, random_state=0)
        df_y_train = pd.DataFrame({'fea0':y[0:train_rows,]})
        df_y_test = pd.DataFrame({'fea0':y[train_rows:,]})

    df_X_train = pd.DataFrame({'fea%d'%i:X[0:train_rows,i] for i in range(X.shape[1])})
    df_X_test = pd.DataFrame({'fea%d'%i:X[train_rows:,i] for i in range(X.shape[1])})

    return df_X_train, df_X_test, df_y_train, df_y_test

In [5]:

from sklearn.metrics import mean_squared_error
def array_equal(a,b,threshold=2e-3,with_sign=True):
    a = to_nparray(a).ravel()
    b = to_nparray(b).ravel()
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    error = mean_squared_error(a,b)
    res = error<threshold
    return res

def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,cudf.DataFrame) or isinstance(x,cudf.Series):
        return x.to_pandas().values
    return x

# Run tests

In [7]:
%%time
nrows = 2**20
ncols = 399

X_train, X_test, y_train, y_test = load_data(nrows,ncols)
print('training data',X_train.shape)
print('training label',y_train.shape)
print('testing data',X_test.shape)
print('testing label',y_test.shape)
print('label',y_test.shape)

use mortgage data
training data (819, 399)
training label (819, 1)
testing data (205, 399)
testing label (205, 1)
label (205, 1)
CPU times: user 4.14 s, sys: 909 ms, total: 5.05 s
Wall time: 4.89 s


In [6]:
%%time
skols = skLinearRegression(fit_intercept=True,
                  normalize=True)
skols.fit(X_train, y_train)

CPU times: user 17.3 s, sys: 2.97 s, total: 20.3 s
Wall time: 5.39 s


In [7]:
%%time
sk_predict = skols.predict(X_test)
error_sk = mean_squared_error(y_test,sk_predict)

CPU times: user 407 ms, sys: 55.1 ms, total: 462 ms
Wall time: 72.6 ms


In [9]:
%%time
X_cudf = cudf.DataFrame.from_pandas(X_train)
X_cudf_test = cudf.DataFrame.from_pandas(X_test)
y_cudf = y_train.values
y_cudf = y_cudf[:,0]
y_cudf = cudf.Series(y_cudf)

CPU times: user 1.34 s, sys: 3.67 ms, total: 1.34 s
Wall time: 1.31 s


In [10]:
from dask_cuml.linear_model import LinearRegression as cumlOLS_dask
from dask.distributed import Client
cluster = LocalCUDACluster(threads_per_worker=1)
client = Client(cluster)
workers = client.has_what().keys()
X_dask_cudf = dask_cudf.from_cudf(X_cudf, npartitions=len(workers)).persist()
X_dask_cudf_test = dask_cudf.from_cudf(X_cudf, npartitions=len(workers)).persist()
y_dask_cudf = dask_cudf.from_cudf(y_cudf, npartitions=len(workers)).persist()

/conda/envs/rapids/lib/python3.7/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)
distributed.scheduler - INFO - Receive client connection: Client-39005466-524e-11e9-8450-0242ac110002
distributed.core - INFO - Starting established connection


In [11]:
lr = cumlOLS_dask()
lr.fit(X_dask_cudf, y_dask_cudf)

distributed.scheduler - INFO - Receive client connection: Client-worker-3f353c10-524e-11e9-8616-0242ac110002
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Receive client connection: Client-worker-3f84891c-524e-11e9-8619-0242ac110002
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Remove client Client-worker-3f353c10-524e-11e9-8616-0242ac110002
distributed.scheduler - INFO - Remove worker tcp://127.0.0.1:41907
distributed.core - INFO - Removing comms to tcp://127.0.0.1:41907
distributed.scheduler - INFO - Close client connection: Client-worker-3f353c10-524e-11e9-8616-0242ac110002
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

distributed.scheduler - INFO - Register tcp://127.0.0.1:46823
distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:46823
distributed.core - INFO - Starting established connection


In [18]:
%%time
cuols = cuLinearRegression(fit_intercept=True,
                  normalize=True,
                  algorithm='eig')
cuols.fit(X_cudf, y_cudf)

CPU times: user 656 ms, sys: 356 ms, total: 1.01 s
Wall time: 1.01 s


In [19]:
%%time
cu_predict = cuols.predict(X_cudf_test).to_array()
error_cu = mean_squared_error(y_test,cu_predict)

CPU times: user 540 ms, sys: 4 ms, total: 544 ms
Wall time: 541 ms


In [20]:
print("SKL MSE(y):")
print(error_sk)
print("CUML MSE(y):")
print(error_cu)

SKL MSE(y):
5.0556037e-07
CUML MSE(y):
1.22318395e-08
